First of all, make sure your environment has installed the latest version of [🤗 Optimum Graphcore](https://github.com/huggingface/optimum-graphcore).

In [1]:
%pip install optimum[graphcore]

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS:

In [3]:
!apt install git-lfs




git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


Let's print out the versions of Transformers and Optimum Graphcore:

In [4]:
import transformers
import optimum.graphcore

print(transformers.__version__)
print(optimum.graphcore.__version__)

4.24.0
0.3.1


Values for machine size and cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

pod_type = os.getenv("GRAPHCORE_POD_TYPE", "pod16")
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/")

# Fine-tuning a model on a multiple choice task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a multiple choice task, which is the task of selecting the most plausible inputs in a given selection. The dataset used here is [SWAG](https://www.aclweb.org/anthology/D18-1009/) but you can adapt the pre-processing to any other multiple choice dataset you like, or your own data. SWAG is a dataset about commonsense reasoning, where each example describes a situation then proposes four options that could go after it. 

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a mutiple choice head and is supported by Optimum Graphcore. The IPU config files of the supported models are available in Graphcore's [Hugging Face account](https://huggingface.co/Graphcore). You can also create your own IPU config file locally. 

In this notebook, we are using both data parallelism and pipeline parallelism (see this [tutorial](https://github.com/graphcore/tutorials/tree/master/tutorials/pytorch/tut2_efficient_data_loading) for more). Therefore the global batch size, which is the actual number of samples used for the weight update, is determined with three factors:
- `global_batch_size = micro_batch_size * gradient_accumulation_steps * replication_factor`

and replication factor is determined by `pod_type`, which will be used as a key to select the replication factor from a dictionary defined in the IPU config file. For example, the dictionary in the IPU config file [Graphcore/gpt2-small-ipu](https://huggingface.co/Graphcore/gpt2-small-ipu/blob/main/ipu_config.json) looks like this:
- `"replication_factor": {"pod4": 1, "pod8": 2, "pod16": 4, "pod32": 8, "pod64": 16, "default": 1}`

Depending on you model and the pod machine you are using, you might need to adjust these three batch-size-related arguments.

In [5]:
model_checkpoint = "roberta-base"

ipu_config_name = "Graphcore/roberta-base-ipu"
micro_batch_size = 2
gradient_accumulation_steps = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the functions `load_dataset`.  

In [6]:
from datasets import load_dataset, load_metric

`load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [7]:
datasets = load_dataset("swag", "regular")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/73546 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20006 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20005 [00:00<?, ? examples/s]

Dataset swag downloaded and prepared to /root/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 73546
    })
    validation: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20006
    })
    test: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20005
    })
})

To access an actual element, you need to select a split first, then give an index:

In [9]:
datasets["train"][0]

{'video-id': 'anetv_jkn6uvmqwh4',
 'fold-ind': '3416',
 'startphrase': 'Members of the procession walk down the street holding small horn brass instruments. A drum line',
 'sent1': 'Members of the procession walk down the street holding small horn brass instruments.',
 'sent2': 'A drum line',
 'gold-source': 'gold',
 'ending0': 'passes by walking down the street playing their instruments.',
 'ending1': 'has heard approaching them.',
 'ending2': "arrives and they're outside dancing and asleep.",
 'ending3': 'turns the lead singer watches the performance.',
 'label': 0}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [10]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(datasets["train"])

,video-id,fold-ind,startphrase,sent1,sent2,gold-source,ending0,ending1,ending2,ending3,label
0,lsmdc1051_Harry_Potter_and_the_goblet_of_fire-7075,1829,He looks down and sees his feet turn into flippers. He,He looks down and sees his feet turn into flippers.,He,gold,grips the reactor into her right hand.,turns back to the zookeeper who was still pointing.,held up his hands as they become webbed.,kicks and pounds the doors.,2
1,lsmdc0008_Fargo-50029,6250,"We are looking over the shoulders of two uniformed policemen who stand on either side of the door, their hands resting lightly on their holstered sidearms. One of them","We are looking over the shoulders of two uniformed policemen who stand on either side of the door, their hands resting lightly on their holstered sidearms.",One of them,gold,takes a pair of clippers and drops the bag into his left palm.,raps at the door.,is preparing in real combat.,has a couple standing on the line.,1
2,lsmdc0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING-61280,535,"Someone moves forward, a fell light in his normally friendly eyes. Someone","Someone moves forward, a fell light in his normally friendly eyes.",Someone,gen,pick up the receiver in her mouth again.,hands him a plank.,leaps up into his passenger seat.,hands the shooter a drinks.,3
3,lsmdc3019_COLOMBIANA-9199,11698,"Spinning, she backhands him with one of the brushes. He","Spinning, she backhands him with one of the brushes.",He,gold,presses a button and closes the door behind her.,drops into someone's bed and runs out of the chamber of a brownstone building.,opens a gift and pokes them with her nervous fingers.,"charges, grabbing her around the waist.",3
4,lsmdc1019_Confessions_Of_A_Shopaholic-80800,9146,"At home, someone runs to the fridge and frantically searches through the freezer. She","At home, someone runs to the fridge and frantically searches through the freezer.",She,gold,fits the remaining bloody balls into the laundry.,brings out a huge block of ice in the middle of which is a credit card.,limps over the crowded hallway towards pastries.,takes the suitcase out of the closet.,1
5,lsmdc3090_YOUNG_ADULT-43871,10867,"Inside a convenience store, she opens a freezer case. Dolce","Inside a convenience store, she opens a freezer case.",Dolce,gen,are both wanting to be pretty.,"to a corner, someone kicks off the shoe.",", someone gives them to her boss then dumps some alcohol into dough.","the slender someone, someone turns on the light.",0
6,lsmdc3017_CHRONICLE-7382,9946,"As our view tilts downward and glides over the asphalt, we glimpse two of the unconscious thugs. Someone","As our view tilts downward and glides over the asphalt, we glimpse two of the unconscious thugs.",Someone,gold,drops someone's hand to her chest.,"telekinetically drags one side - to - side, leaving bloody smears on the pavement.",sidles to a table next to him.,eyes the glamorous on the bed.,1
7,lsmdc3085_TRUE_GRIT-40869,16709,"Silhouetted against the pale sky, someone gallops on across the dusty plain. Someone","Silhouetted against the pale sky, someone gallops on across the dusty plain.",Someone,gold,glances up at someone who hauls himself back down.,soars up the chimney and lands halfway near the closed gate.,"glances at the setting sun, her face cast in its golden glow.","treads toward the lighthouse, which lights up on a second tower.",2
8,lsmdc3006_A_GOOD_DAY_TO_DIE_HARD-2947,3163,"Someone checks the gun's chamber and clip, then holsters it in his waistband. Someone","Someone checks the gun's chamber and clip, then holsters it in his waistband.",Someone,gold,"40, someone slips off a radio.","steps the agent, tosses his gun up, and points the gun into the direction of the other.",gradually combines to strip the mechanism tasteful.,shifts his exasperated gaze as his son helps someone to his feet and urges him onward.,3
9,anetv_kfwwya1qzXM,5772,He describes the chords of a song and the proper finger placement to form the chords with a fingering chord 

Each example in the dataset has a context composed of a first sentence (in the field `sent1`) and an introduction to the second sentence (in the field `sent2`). Then four possible endings are given (in the fields `ending0`, `ending1`, `ending2` and `ending3`) and the model must pick the right one (indicated in the field `label`). The following function lets us visualize a give example a bit better:

In [12]:
def show_one(example):
    print(f"Context: {example['sent1']}")
    print(f"  A - {example['sent2']} {example['ending0']}")
    print(f"  B - {example['sent2']} {example['ending1']}")
    print(f"  C - {example['sent2']} {example['ending2']}")
    print(f"  D - {example['sent2']} {example['ending3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

In [13]:
show_one(datasets["train"][0])

Context: Members of the procession walk down the street holding small horn brass instruments.
  A - A drum line passes by walking down the street playing their instruments.
  B - A drum line has heard approaching them.
  C - A drum line arrives and they're outside dancing and asleep.
  D - A drum line turns the lead singer watches the performance.

Ground truth: option A


In [14]:
show_one(datasets["train"][15])

Context: Now it's someone's turn to rain blades on his opponent.
  A - Someone pats his shoulder and spins wildly.
  B - Someone lunges forward through the window.
  C - Someone falls to the ground.
  D - Someone rolls up his fast run from the water and tosses in the sky.

Ground truth: option C


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [15]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [16]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2, 2, 2409, 42, 3645, 1411, 19, 24, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

We can them write the function that will preprocess our samples. The tricky part is to put all the possible pairs of sentences in two big lists before passing them to the tokenizer, then un-flatten the result so that each example has four input ids, attentions masks, etc.

We also define a maximum sequence length (`max_seq_length`) to either pad or truncate our samples to so that each sample is the same size. Then, when calling the `tokenizer`, we use the arguments `truncation=True`, `padding="max_length"` and `max_length=max_seq_length`.

In [17]:
ending_names = ["ending0", "ending1", "ending2", "ending3"]
max_seq_length = 128

def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    # Grab all second sentences possible for each context.
    question_headers = examples["sent2"]
    second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize
    tokenized_examples = tokenizer(
        first_sentences, 
        second_sentences, 
        truncation=True,
        max_length=max_seq_length,
        padding="max_length",
    )
    # Un-flatten
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists of lists for each key: a list of all examples (here 5), then a list of all choices (4) and a list of input IDs:

In [18]:
examples = datasets["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 4 [128, 128, 128, 128]


To check we didn't do anything group when grouping all possibilites then unflattening, let's have a look at the decoded inputs for a given example:

In [19]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

['<s>A drum line passes by walking down the street playing their instruments.</s></s>Members of the procession are playing ping pong and celebrating one left each in quick.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s>A drum line passes by walking down the street playing their instruments.</s></s>Members of the procession wait slowly towards the cadets.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

We can compare it to the ground truth:

In [20]:
show_one(datasets["train"][3])

Context: A drum line passes by walking down the street playing their instruments.
  A - Members of the procession are playing ping pong and celebrating one left each in quick.
  B - Members of the procession wait slowly towards the cadets.
  C - Members of the procession makes a square call and ends by jumping down into snowy streets where fans begin to take their positions.
  D - Members of the procession play and go back and forth hitting the drums while the audience claps for them.

Ground truth: option D


This seems alright, so we can apply this function on all the examples in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [21]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

  0%|          | 0/74 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our task is about mutliple choice, we use the `AutoModelForMultipleChoice` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. We also define the `IPUConfig`, which is a class that specifies attributes and configuration parameters to compile and put the model on the device. We initialize it with one config name or path, which we set earlier.

In [22]:
from transformers import AutoModelForMultipleChoice
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)
ipu_config = IPUConfig.from_pretrained(ipu_config_name, executable_cache_dir=executable_cache_dir)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultipleChoice: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `IPUTrainer`, we will need to define a few more things. The most important is the [`IPUTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [23]:
model_name = model_checkpoint.split("/")[-1]
args = IPUTrainingArguments(
    f"{model_name}-finetuned-swag",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    pod_type=pod_type,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    dataloader_drop_last=True,
    push_to_hub=False,
    # hub_model_id = f"username-or-organization/{model_name}-finetuned-squad"
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate and use the gloal batch size defined at the top of the notebook through `micro_batch_size`, `gradient_accumulation_steps` and `pod_type`. We also customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-swag"` or `"huggingface/bert-finetuned-swag"`).

The last thing to define for our `IPUTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits:

In [24]:
import numpy as np

def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

Then we just need to pass all of this along with our datasets to the `IPUTrainer`:

In [25]:
from transformers import default_data_collator

trainer = IPUTrainer(
    model,
    ipu_config,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

Overriding IPU config: gradient_accumulation_steps=16
-------------------- Device Allocation --------------------
Embedding  --> IPU 0
Encoder 0  --> IPU 1
Encoder 1  --> IPU 1
Encoder 2  --> IPU 1
Encoder 3  --> IPU 1
Encoder 4  --> IPU 2
Encoder 5  --> IPU 2
Encoder 6  --> IPU 2
Encoder 7  --> IPU 2
Encoder 8  --> IPU 3
Encoder 9  --> IPU 3
Encoder 10 --> IPU 3
Encoder 11 --> IPU 3
Classifier Output --> IPU 3
-----------------------------------------------------------


We can now finetune our model by just calling the `train` method:

In [26]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `PipelinedRobertaForMultipleChoice.forward` and have been ignored: sent2, gold-source, fold-ind, ending1, video-id, ending3, ending2, sent1, ending0, startphrase.
Compiling Model...
Graph compilation: 100%|██████████| 100/100 [05:11<00:00]
Compiled/Loaded model in 347.5222702920437 secs
***** Running training *****
  Num examples = 73546
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Device Iterations = 1
  Replication Factor = 4
  Gradient Accumulation steps = 16
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Total optimization steps = 1722


  0%|          | 0/1722 [00:00<?, ?it/s]

{'loss': 1.3892, 'learning_rate': 4.9709639953542396e-05, 'epoch': 0.02}
{'loss': 1.1033, 'learning_rate': 4.9419279907084783e-05, 'epoch': 0.03}
{'loss': 1.1104, 'learning_rate': 4.9128919860627184e-05, 'epoch': 0.05}
{'loss': 0.9539, 'learning_rate': 4.883855981416957e-05, 'epoch': 0.07}
{'loss': 1.0036, 'learning_rate': 4.8548199767711965e-05, 'epoch': 0.09}
{'loss': 1.003, 'learning_rate': 4.825783972125435e-05, 'epoch': 0.1}
{'loss': 0.8866, 'learning_rate': 4.796747967479675e-05, 'epoch': 0.12}
{'loss': 0.9367, 'learning_rate': 4.767711962833915e-05, 'epoch': 0.14}
{'loss': 0.8064, 'learning_rate': 4.7386759581881534e-05, 'epoch': 0.16}
{'loss': 0.8734, 'learning_rate': 4.709639953542393e-05, 'epoch': 0.17}
{'loss': 0.7134, 'learning_rate': 4.680603948896632e-05, 'epoch': 0.19}
{'loss': 0.9928, 'learning_rate': 4.6515679442508716e-05, 'epoch': 0.21}
{'loss': 0.8884, 'learning_rate': 4.62253193960511e-05, 'epoch': 0.23}
{'loss': 0.8715, 'learning_rate': 4.59349593495935e-05, 'epoc

Saving model checkpoint to roberta-base-finetuned-swag/checkpoint-500


{'loss': 0.7158, 'learning_rate': 3.548199767711963e-05, 'epoch': 0.87}


...

The following columns in the evaluation set  don't have a corresponding argument in `PoptorchPipelinedRobertaForMultipleChoice.forward` and have been ignored: sent2, gold-source, fold-ind, ending1, video-id, ending3, ending2, sent1, ending0, startphrase.
***** Running Evaluation *****
  Num examples = 20006
  Batch size = 40


  0%|          | 0/500 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.44482421875, 'eval_accuracy': 0.8381500244140625, 'eval_runtime': 20.0499, 'eval_samples_per_second': 997.513, 'eval_steps_per_second': 24.938, 'epoch': 3.0}
{'train_runtime': 833.1948, 'train_samples_per_second': 264.543, 'train_steps_per_second': 2.067, 'train_loss': 0.5913507902386851, 'epoch': 3.0}


TrainOutput(global_step=1722, training_loss=0.5913507902386851, metrics={'train_runtime': 833.1948, 'train_samples_per_second': 264.543, 'train_steps_per_second': 2.067, 'train_loss': 0.5913507902386851, 'epoch': 3.0})

We can check how our model did with the `evaluate` method:

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `PoptorchPipelinedRobertaForMultipleChoice.forward` and have been ignored: sent2, gold-source, fold-ind, ending1, video-id, ending3, ending2, sent1, ending0, startphrase.


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMultipleChoice

model = AutoModelForMultipleChoice.from_pretrained("sgugger/my-awesome-model")
```